In [1]:
import os
import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import neurokit2 as nk

In [2]:
def load_data(dir):
    data = []
    order = []
    for file in os.listdir(dir):
        if file.endswith(".csv"):
            print(os.path.join(dir, file))
            df = pd.read_csv(os.path.join(dir, file),on_bad_lines='skip')
            if not df.empty:
                order.append(int(file[:-4]))
                data.append(df["DATA"])
    return data, order


try:
    Group1, Group1_order = load_data("/mnt/ArtHeart/Data/Group 1")
    Group2, Group2_order = load_data("/mnt/ArtHeart/Data/Group 2")
    Group3, Group3_order = load_data("/mnt/ArtHeart/Data/Group 3")
except:
    try:
        Group1, Group1_order = load_data("Data/Group 1")
        Group2, Group2_order = load_data("Data/Group 2")
        Group3, Group3_order = load_data("Data/Group 3")
    except:
        Group1, Group1_order = load_data("W:\PhD\ArtHeart\Data\Group 1")
        Group2, Group2_order = load_data("W:\PhD\ArtHeart\Data\Group 2")
        Group3, Group3_order = load_data("W:\PhD\ArtHeart\Data\Group 3")

Data/Group 1/6.csv
Data/Group 1/40.csv
Data/Group 1/7.csv
Data/Group 1/5.csv
Data/Group 1/4.csv
Data/Group 1/1.csv
Data/Group 1/3.csv
Data/Group 1/2.csv
Data/Group 1/37.csv
Data/Group 1/23.csv
Data/Group 1/22.csv
Data/Group 1/36.csv
Data/Group 1/20.csv
Data/Group 1/34.csv
Data/Group 1/35.csv
Data/Group 1/21.csv
Data/Group 1/25.csv
Data/Group 1/31.csv
Data/Group 1/19.csv
Data/Group 1/18.csv
Data/Group 1/30.csv
Data/Group 1/24.csv
Data/Group 1/32.csv
Data/Group 1/26.csv
Data/Group 1/27.csv
Data/Group 1/33.csv
Data/Group 1/16.csv
Data/Group 1/17.csv
Data/Group 1/29.csv
Data/Group 1/15.csv
Data/Group 1/14.csv
Data/Group 1/28.csv
Data/Group 1/10.csv
Data/Group 1/38.csv
Data/Group 1/39.csv
Data/Group 1/11.csv
Data/Group 1/13.csv
Data/Group 1/12.csv
Data/Group 1/9.csv
Data/Group 1/8.csv
Data/Group 2/6.csv
Data/Group 2/40.csv
Data/Group 2/7.csv
Data/Group 2/5.csv
Data/Group 2/4.csv
Data/Group 2/42.csv
Data/Group 2/46.csv
Data/Group 2/1.csv
Data/Group 2/47.csv
Data/Group 2/3.csv
Data/Group 2/44

In [3]:
Group1_scores = []
Group2_scores = []
Group3_scores = []

def emotion_definer(emotions):
    emotions = emotions.split(";")
    emotions = [x.strip(' ') for x in emotions]
    emotion_score = []
    for emotion in emotions:
        emotion = emotion.lower()
        if emotion == 'happy' or emotion == 'immersed' or emotion == 'excited' or emotion == 'hyped up' or emotion == 'hypedup' or emotion == 'delighted':
            emotion_score.append(2)
        elif emotion == 'sad' or emotion == 'angry' or ('frustrated' in emotion) or ('annoyed' in emotion) or emotion == 'bored' or emotion == 'sleepy' or emotion == 'tired' or emotion == 'nervous' or ('anxious' in emotion) or emotion == 'scared':
            emotion_score.append(0)
        elif emotion == 'tense' or emotion == 'confused' or  emotion == 'relaxed' or emotion == 'intrigued' or emotion == 'interest' or emotion == 'content' or emotion == 'calm' or emotion == 'mellow' or emotion == 'alright' or emotion == 'fine' or emotion == 'other':
            emotion_score.append(1)

    if not emotion_score:
        emotion_score.append(0)
    
    emotion = round(np.mean(emotion_score))
    
    if emotion is np.nan:
        emotion = 0

    return emotion
        
        
        
    
def load_Scores(location):
    print(location)
    folder = location[:-19]
    print(folder)
    var = os.listdir(folder)
    group = ''

    if "Group 1" in location:
        group = "Group 1"
    elif "Group 2" in location:
        group = "Group 2"
    elif "Group 3" in location:
        group = "Group 3"
        folder = folder + "/"

    for file in var:
        if group in file:
            if "xlsx" in file:
                location = folder + file
                data = pd.read_excel(location)
            elif "csv" in file:
                location = folder + file
                data = pd.read_csv(location)


    if "How would you rate this performance?" in data.columns:
        order = data["Watch ID"]
        scores = data["How would you rate this performance?"]
    elif "How did you rate the performance?" in data.columns:
        order = data["Watch ID"]
        scores  = data["How did you rate the performance?"]
    elif "How would you rate this performance?（10 is the best）" in data.columns:
        order = data["ID"]
        scores  = data["How would you rate this performance?（10 is the best）"]

    if "How would you describe your feelings through the performance?" in data.columns:
        emotions = data["How would you describe your feelings through the performance?"]
    elif "How do you describe your feeling during the programme? You may make multiple choices.\n" in data.columns:
        emotions = data["How do you describe your feeling during the programme? You may make multiple choices.\n"]
        
    emotions = emotions.apply(emotion_definer)
        
    return scores, order, emotions


def match_scores_to_data(scores, scores_order, emotions, data, data_order):
    matched_scores = []
    matched_data = []
    matched_emotions = []

    # Zip the scores and order and the data and data order
    scores_zip = zip(scores_order, scores, emotions)
    data_zip = zip(data_order, data)

    # Sort the zipped lists by order
    scores_zip = sorted(scores_zip, key=lambda x: x[0])
    data_zip = sorted(data_zip, key=lambda x: x[0])

    # Unzip the sorted lists
    sorted_scores_order, sorted_scores, sorted_emotions = zip(*scores_zip)
    sorted_data_order, sorted_data = zip(*data_zip)

    # Match the scores to the data using the order of each
    for i in range(min(len(sorted_scores_order), len(sorted_data_order))):
        if sorted_scores_order[i] == sorted_data_order[i]:
            matched_scores.append(sorted_scores[i])
            matched_data.append(sorted_data[i])
            matched_emotions.append(emotions[i])
    return matched_scores, matched_data, matched_emotions



try:
    Group1_scores, Group1_scores_order, Group1_emotions = load_Scores("/mnt/ArtHeart/Data/Group 1 Scores.xlsx")
    Group2_scores,Group2_scores_order, Group2_emotions = load_Scores("/mnt/ArtHeart/Data/Group 2 Scores.xlsx")
    Group3_scores,Group3_scores_order, Group3_emotions = load_Scores("/mnt/ArtHeart/Data/Group 3 Scores.csv")
except:
    try:
        Group1_scores, Group1_scores_order, Group1_emotions = load_Scores("W:\PhD\ArtHeart\Data\Group 1 Scores.xlsx")
        Group2_scores,Group2_scores_order, Group2_emotions = load_Scores("W:\PhD\ArtHeart\Data\Group 2 Scores.xlsx")
        Group3_scores,Group3_scores_order, Group3_emotions = load_Scores("W:\PhD\ArtHeart\Data\Group 3 Scores.csv")

    except:
        Group1_scores, Group1_scores_order, Group1_emotions = load_Scores("Data/Group 1 Scores.xlsx")
        Group2_scores,Group2_scores_order, Group2_emotions = load_Scores("Data/Group 2 Scores.xlsx")
        Group3_scores,Group3_scores_order, Group3_emotions = load_Scores("Data/Group 3 Scores.csv")





#Group1_scores, Group1_processed, Group1_emotions = match_scores_to_data(Group1_scores, Group1_scores_order, Group1_emotions, Group1_processed, Group1_order)
Group2_scores, Group2_processed, Group2_emotions = match_scores_to_data(Group2_scores, Group2_scores_order, Group2_emotions, Group2, Group2_order)
Group3_scores, Group3_processed, Group3_emotions = match_scores_to_data(Group3_scores, Group3_scores_order, Group3_emotions, Group3, Group3_order)


/mnt/ArtHeart/Data/Group 1 Scores.xlsx
/mnt/ArtHeart/Data/
W:\PhD\ArtHeart\Data\Group 1 Scores.xlsx
W:\PhD\ArtHeart\Data\
Data/Group 1 Scores.xlsx
Data/
Data/Group 2 Scores.xlsx
Data/
Data/Group 3 Scores.csv
Data


In [4]:
print("Group 2: ", len(Group2_scores), len(Group2_processed), len(Group2_emotions))
print("Group 3: ", len(Group3_scores), len(Group3_processed), len(Group3_emotions))

print(Group2_processed[0])

Group 2:  30 30 30
Group 3:  30 30 30
0         1234
1         1233
2         1233
3         1233
4         1233
          ... 
127579      59
127580      59
127581      59
127582      59
127583      59
Name: DATA, Length: 127584, dtype: int64


In [5]:
print(Group2_emotions)
print(Group3_emotions)

[1, 2, 2, 2, 1, 0, 1, 2, 2, 1, 1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1]
[1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 0, 0, 2, 2, 0, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2]


In [6]:
from tensorflow import one_hot
from sklearn.preprocessing import StandardScaler

X = []
X.extend(Group2_processed)
X.extend(Group3_processed)

scale = StandardScaler()

for i in range(len(X)):
    X[i] = np.array(X[i]).reshape(-1,1)
    X[i] = scale.fit_transform(X[i])

max_len = 0
for seq in X:
    if len(seq) > max_len:
        max_len = len(seq)

for i in range(len(X)):
    seq_len = len(X[i])
    if seq_len < max_len:
        padding_len = max_len - seq_len
        padding = [-10] * padding_len
        series = X[i].flatten()
        series = np.concatenate((series, padding), axis=0)
        X[i] = series.flatten()
    else:
        X[i] = X[i].flatten()

y = []
y.extend(Group2_emotions)
y.extend(Group3_emotions)



X = np.array(X).reshape(-1, max_len, 1)
y = np.array(y)
print(X.shape, y.shape)



(60, 129624, 1) (60,)


In [7]:
from sklearn.model_selection import train_test_split

# Split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = one_hot(y_train, 3)
y_test = one_hot(y_test, 3)


2023-06-20 13:12:48.044833: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-06-20 13:12:48.044872: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-06-20 13:12:48.044915: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-06-20 13:12:48.045541: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-20 13:12:48.045898: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Masking
from tensorflow.keras.models import Model

model = tf.keras.Sequential()
model.add(Masking(mask_value=-10, input_shape=(None, 1)))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, None, 1)           0         
                                                                 
 lstm (LSTM)                 (None, None, 64)          16896     
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 50115 (195.76 KB)
Trainable params: 50115 

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=10, verbose=1, steps_per_epoch=10)

Epoch 1/10


2023-06-20 13:12:50.667621: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-20 13:12:51.075457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-20 13:12:58.063125: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_40/output/_23'
2023-06-20 13:1